In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install --upgrade -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# caustique d'onde

In [3]:
import os
import numpy as np
import MotionClouds as mc
from caustique import init, Caustique, figpath

In [4]:
print(f'Saving our simulations in={figpath}')

Saving our simulations in=2022-07-25_caustique


In [5]:
opt = init()

In [6]:
opt

Namespace(tag='caustique', figpath='2022-07-25_caustique', vext='mp4', nx=1024, ny=1024, nframe=640, bin_dens=2, bin_spectrum=6, seed=42, H=10.0, variation=0.09, sf_0=0.004, B_sf=0.002, V_Y=0.3, V_X=0.3, B_V=4.0, theta=2.399988291783386, B_theta=1.0471975511965976, min_lum=0.2, fps=18, multispectral=True, cache=False, verbose=False)

# une simple caustique

## a wave pattern

In [ ]:
if not os.path.isfile(f'{opt.figpath}/{opt.tag}{mc.vext}'):
    print('Doing', f'{opt.figpath}/{opt.tag}{mc.vext}')
    c = Caustique(opt)
    z = c.wave()
    mc.anim_save(z.swapaxes(0, 1), f'{opt.figpath}/{opt.tag}')
# mc.in_show_video(f'{opt.tag}', figpath=opt.figpath)

Doing 2022-07-25_caustique/caustique.mp4


## generating caustics

In [ ]:
c = Caustique(opt)
z = c.wave()
z.shape

In [ ]:
N_wavelengths = len(c.cs_srgb.cmf[:, 0])
N_wavelengths, N_wavelengths//c.opt.bin_spectrum

In [ ]:
for ii, i in enumerate(range(0, N_wavelengths, c.opt.bin_spectrum)): print(ii, i)

In [ ]:
videoname = f'{opt.figpath}/{opt.tag}.{opt.vext}'
if not os.path.isfile(videoname):
    c = Caustique(opt)
    z = c.wave()
    
    videoname = c.plot(z)
    

In [ ]:
c.show(videoname)

In [ ]:
z.shape

## exploring parameters

In [ ]:
N_scan = 9
base = 2

In [ ]:
opt = init()
opt.figpath = figpath

c = Caustique(opt)
# compute just once
z = c.wave()

for H_ in c.opt.H*np.logspace(-1, 1, N_scan, base=base):
    opt = init()
    opt.figpath = figpath
    c = Caustique(opt)

    print(f'H = {H_:.3f}')
    c.opt.H = H_
    c.opt.tag = f'{opt.tag}_H_{H_:.3f}'
    videoname = f'{opt.figpath}/{c.opt.tag}.{opt.vext}'
    if not os.path.isfile(videoname):
        url=c.plot(z, videoname=videoname)
    c.show(videoname)

In [ ]:
opt = init()
opt.figpath = figpath

c = Caustique(opt)
# compute just once
z = c.wave()

for variation_ in c.opt.variation*np.logspace(-1, 1, N_scan, base=10):
    opt = init()
    opt.figpath = figpath
    c = Caustique(opt)

    print(f'variation = {variation_:.3f}')
    c.opt.variation = variation_
    c.opt.tag = f'{opt.tag}_variation_{variation_:.3f}'
    videoname = f'{opt.figpath}/{c.opt.tag}.{opt.vext}'
    if not os.path.isfile(videoname):
        url=c.plot(z, videoname=videoname)
    c.show(videoname)

In [ ]:
for variable in ['sf_0', 'B_sf', 'theta', 'B_theta', 'V_X', 'V_Y', 'B_V']:
    print(f'======{variable}======')
    for modul in np.logspace(-1, 1, N_scan, base=base):
        opt = init()
        opt.figpath = figpath

        c = Caustique(opt)
        c.d[variable] *= modul
        c.opt.tag = f'{opt.tag}_{variable}_modul_{modul:.3f}'
        videoname = f'{opt.figpath}/{c.opt.tag}.{opt.vext}'

        print(f'{variable}={variable}(default)*{modul:.3f}={c.d[variable]:.3E}')
        if not os.path.isfile(videoname):
            print('Doing ', videoname)
            z = c.wave()
            mcname = f'{opt.figpath}/{c.opt.tag}'
            if not os.path.isfile(f'{mcname}{mc.vext}'): 
                print('Doing ', f'{mcname}{mc.vext}')
                mc.anim_save(z.swapaxes(0, 1), f'{mcname}')
            url=c.plot(z, videoname=videoname)
        c.show(videoname)